# <font color="darkblue"> Prática 04: Random Forest </font>

**Objetivos:**


*   Comparar os resultados da Árvore de Decisão induzida por dados com o algoritmo de *Random Forest*;
*   Implementar o *tunning* do algoritmo de *Random Forest*.  

**Requisitos de execução:**


*   Upload do arquivo *diabetes.csv*

**Atividade 1:**

1. Visitar a base de dados: https://www.kaggle.com/uciml/pima-indians-diabetes-database
2. Carregar os dados do arquivo *diabetes.csv* utilizando o pandas.

    

In [ ]:
import pandas as pd 

diabete = pd.read_csv("diabetes.csv", sep=',')

# Pega o cabecalho do arquivo
print(diabete.head())

**Atividade 2:**

1. Extrair os valores do *DataFrame* pandas e dividir em dados de treino e teste.


In [2]:
from sklearn.model_selection import train_test_split

Features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',  'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
X = diabete[Features].values
y = diabete.Outcome.values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2698)

print("Tamanho treino: " + str(len(x_train)))
print("Tamanho teste: " + str(len(x_test)))


Tamanho treino: 614
Tamanho teste: 154


**Atividade 3:**

1. Utilize a classe *DecisionTreeClassifier*, importada do pacote *sklearn.tree*, para inferir aprendizado dos dados de treinamento;
2. Compute as métricas de aprendizado sobre os dados de teste.

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

print('Ein: %0.4f' % (1 - accuracy_score(y_train, clf.predict(x_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, clf.predict(x_test))))
print(classification_report(y_test, clf.predict(x_test)))

Ein: 0.0000
Eout: 0.3442
              precision    recall  f1-score   support

           0       0.73      0.72      0.73        98
           1       0.53      0.54      0.53        56

    accuracy                           0.66       154
   macro avg       0.63      0.63      0.63       154
weighted avg       0.66      0.66      0.66       154



**Atividade 4:**


1. Utilize a classe *RandomForestClassifier*, importada do pacote *sklearn.ensemble*, para inferir aprendizado dos dados de treinamento;
2. Compute as métricas de aprendizado sobre os dados de teste e compare com os resultados da atividade anterior.

Parâmetros:


*   *n_estimators* : número de árvore aleatórias inferidas
*   *max_depth* : profundidade máxima da árvore



In [4]:
from sklearn.ensemble import RandomForestClassifier

# Inicializa 100 arvores aleatórias
rf = RandomForestClassifier(n_estimators = 50, max_depth = 20, random_state = 42)

rf.fit(x_train, y_train)

print('Ein: %0.4f' % (1 - accuracy_score(y_train, rf.predict(x_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, rf.predict(x_test))))
print(classification_report(y_test, rf.predict(x_test)))

Ein: 0.0000
Eout: 0.2922
              precision    recall  f1-score   support

           0       0.73      0.87      0.79        98
           1       0.65      0.43      0.52        56

    accuracy                           0.71       154
   macro avg       0.69      0.65      0.65       154
weighted avg       0.70      0.71      0.69       154



**Atividade 5:**

1. Implemente os valores dos hiperparâmetros que serão combinados durantea a fase de validação

Parâmetros:

*   *max_features* : número de características a serem consideradas em cada divisão da árvore



In [12]:
import numpy as np
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 70, num = 6)]
# Number of features to consider at every split
max_features = ['sqrt', len(X[0])]
max_depth = [2, 5, 7, 10, 20, 30]
max_depth.append(None)

# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              }
pprint(param_grid)

{'max_depth': [2, 5, 7, 10, 20, 30, None],
 'max_features': ['sqrt', 8],
 'n_estimators': [20, 30, 40, 50, 60, 70]}


Atividade 6:

1. Implementar a validação da Random Forest através da classe *GridSearchCV* do pacote *sklearn.model_selection*;
2. Imprimir os parâmetros do melhor classificador;
3. Imprimir os $E_{in}$ e $E_{out}$  e as métricas de aprendizado.

Parâmetros:


*   *estimator* : instância do classificador cujos hiperparâmetros serão analisados;
*   *cv* : número de divisões do conjunto de treinamento para ser usado na técnica de validação cruzada (10 é um bom valor observado na prática);
*   *param_grid* : conjunto de parâmetros a serem combinados durante a fase de validação.



In [13]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()

CV_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv = 5, verbose=2, n_jobs=-1)

# Fit the random search model
CV_rf.fit(x_train, y_train)

print(CV_rf.best_estimator_)


print('Ein: %0.4f' % (1 - accuracy_score(y_train, CV_rf.predict(x_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, CV_rf.predict(x_test))))
print(classification_report(y_test, CV_rf.predict(x_test)))



Fitting 5 folds for each of 84 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 420 out of 420 | elapsed:   34.0s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=70,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Ein: 0.0603
Eout: 0.2792
              precision    recall  f1-score   support

           0       0.75      0.84      0.79        98
           1       0.64      0.52      0.57        56

    accuracy                           0.72       154
   macro avg       0.70      0.68      0.68       154
weighted avg       0.71      0.72      0.71       154



Atividade 7:

1. Implementar a validação da Random Forest através da classe *RandomizedSearchCV* do pacote *sklearn.model_selection*;
2. Imprimir os parâmetros do melhor classificador;
3. Imprimir os $E_{in}$ e $E_{out}$ e as métricas de aprendizado.

Parâmetros:


*   *n_iter* : número aleatório de combinações dos parâmetros que serão validadas.



In [7]:
from sklearn.model_selection import RandomizedSearchCV


rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, n_iter = 25, cv = 5, verbose=2, n_jobs=-1, random_state=42)

# Fit the random search model
rf_random.fit(x_train, y_train)

print(rf_random.best_estimator_)

print('Ein: %0.4f' % (1 - accuracy_score(y_train, rf_random.predict(x_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, rf_random.predict(x_test))))
print(classification_report(y_test, rf_random.predict(x_test)))

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 122 out of 125 | elapsed:    9.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    9.4s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=40, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Ein: 0.0000
Eout: 0.2857
              precision    recall  f1-score   support

           0       0.75      0.84      0.79        98
           1       0.64      0.50      0.56        56

    accuracy                           0.71       154
   macro avg       0.69      0.67      0.67       154
weighted avg       0.71      0.71      0.71       154

